In [1]:
import ee
import geemap
import ipywidgets as widgets
from IPython.display import display
import datetime
import json

# Authenticate and initialize Google Earth Engine API
ee.Authenticate()
ee.Initialize(project='ee-geosynta')

In [2]:
# Initialize Earth Engine
ee.Initialize()

# Define the region of interest (ROI)
IND = ee.Geometry.Polygon(
         [[65.05105785032507, 37.597177512338746],
          [65.05105785032507, 5.507083786578366],
          [97.30691722532507, 5.507083786578366],
          [97.30691722532507, 37.597177512338746]])

# Define visualization parameters
ls_true_color_vis = {
    'min': 0.0,
    'max': 0.4,
}
ls_ndvi_vis = {
    'min': -0.1,
    'max': 0.9,
    'palette': ['blue', 'white', 'green']
}
md_true_color_vis = {
    'min': -100.0,
    'max': 8000.0,
}
md_ndvi_vis = {
    'min': -0.1,
    'max': 0.9,
    'palette': ['blue', 'white', 'green']
}

# Create the map
m = geemap.Map()
m.set_center(79.956, 22.377, 5)

# Create widgets
zoom_slider = widgets.IntSlider(min=1, max=15, value=m.zoom, description='Zoom')
year_range = widgets.IntRangeSlider(value=[2017, 2017], 
                                    min=2014, 
                                    max=datetime.datetime.now().year, 
                                    description='Year Range')

reset_button = widgets.Button(description='Reset', 
                              style=widgets.ButtonStyle(button_color='#e32d2d'),
                              layout=widgets.Layout(width='100px'))

# Define function to update map based on year range
def update_map(change):
    # Get selected year range
    start_year, end_year = year_range.value
    
    # Filter LANDSAT dataset based on year range
    LANDSAT = ee.ImageCollection('LANDSAT/LC08/C02/T1_TOA')\
                .filterDate(f'{start_year}-01-01', f'{end_year}-12-31')\
                .filterBounds(IND)
    ls_true_color_432 = LANDSAT.select(['B4', 'B3', 'B2']).median()
    ls_ndvi = LANDSAT.map(lambda img: img.normalizedDifference(['B5', 'B4']).rename('NDVI')).median()
    
    # Filter MODIS dataset based on year range
    MODIS = ee.ImageCollection('MODIS/061/MOD09GA')\
                  .filterDate(f'{start_year}-01-01', f'{end_year}-12-31')\
                  .filterBounds(IND)
    md_true_color_143 = MODIS.select(['sur_refl_b01', 'sur_refl_b04', 'sur_refl_b03']).median()
    md_ndvi = MODIS.map(lambda img: img.normalizedDifference(['sur_refl_b02', 'sur_refl_b01']).rename('NDVI')).median()
    
    # Check if layers exist before removing them
    if 'LANDSAT True Color (432)' in m.ee_layers:
        m.remove_layer('LANDSAT True Color (432)')
    if 'LANDSAT NDVI' in m.ee_layers:
        m.remove_layer('LANDSAT NDVI')
    if 'MODIS True Color (143)' in m.ee_layers:
        m.remove_layer('MODIS True Color (143)')
    if 'MODIS NDVI' in m.ee_layers:
        m.remove_layer('MODIS NDVI')
    
    # Add updated layers
    m.add_layer(ls_true_color_432, ls_true_color_vis, 'LANDSAT True Color (432)')
    m.add_layer(ls_ndvi, ls_ndvi_vis, 'LANDSAT NDVI')
    m.add_layer(md_true_color_143, md_true_color_vis, 'MODIS True Color (143)')
    m.add_layer(md_ndvi, md_ndvi_vis, 'MODIS NDVI')

    # Update the map display
    m

# Attach update function to year_range slider
year_range.observe(update_map, names='value')

# Define reset functionality
def reset_widgets(b):
    m.set_center(79.956, 22.377, 5)  # Reset map center
    zoom_slider.value = 5  # Reset zoom to 5
    year_range.value = [2017, 2017]  # Reset year range to 2017
    update_map(None)  # Update map with default year range

reset_button.on_click(reset_widgets)

# Update zoom slider
def update_zoom(change):
    m.set_zoom(change.new)

zoom_slider.observe(update_zoom, names='value')

# Display widgets and map
display(widgets.HBox([zoom_slider, reset_button], 
                     layout=widgets.Layout(justify_content='space-between', width='100%')))
display(year_range)
display(m)

# Initial map update
update_map(None)


IntRangeSlider(value=(2017, 2017), description='Year Range', max=2024, min=2014)

Map(center=[22.377, 79.956], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…

No such comm: 009bb25432f14720829cf473c86997cb
No such comm: f3a8d82db8ee4e0cb183708b09711a73
No such comm: 366ab793384f444d94ad7abf6b3cac82
No such comm: f3a8d82db8ee4e0cb183708b09711a73
No such comm: 0ae537b6677248e9a6af691333b0a8bb
No such comm: 4b864e4abbef428e9dfb36bf3b206c3c
